In [2]:
import json
from pathlib import Path
from soundevent import data, io
from collections import defaultdict
from tensorflow._api.v2.lite import Interpreter
from audioclass import SimpleIterator
from audioclass.models.birdnet import BirdNET
import pandas as pd
from tqdm import tqdm

In [ ]:
# Load in the annotation project

In [3]:
gn_annotation_path = Path("SM2_6S_2024-07-11T00_18_21.016729.json")
with open(gn_annotation_path) as file:
    annotation_contents = json.load(file)

print(json.dumps(annotation_contents, indent=2))

{
  "version": "1.1.0",
  "created_on": "2024-07-11T00:18:21.016729",
  "data": {
    "uuid": "b35df4fb-caa0-4697-b389-6d5d3ba8c27c",
    "collection_type": "annotation_project",
    "users": [
      {
        "uuid": "63e6952b-1e95-440b-9fbb-6e0aa03b4ecb",
        "username": "hhhyt",
        "email": "yuting.huang.23@ucl.ac.uk",
        "name": "Yuting Huang",
        "institution": null
      }
    ],
    "tags": [
      {
        "id": 0,
        "key": "n/a",
        "value": "nothing"
      },
      {
        "id": 1,
        "key": "environment",
        "value": "rain or wind"
      },
      {
        "id": 2,
        "key": "sounds",
        "value": "parrot"
      },
      {
        "id": 3,
        "key": "sound",
        "value": "unknown"
      },
      {
        "id": 4,
        "key": "sound",
        "value": "bird"
      },
      {
        "id": 5,
        "key": "parrot",
        "value": "whistle"
      },
      {
        "id": 6,
        "key": "sound",
        "val

In [4]:
gn_sample = io.load(gn_annotation_path, type="annotation_project")
print(repr(gn_sample))

AnnotationProject(created_on=datetime.datetime(2024, 5, 30, 15, 24, 21, 734859), name='SM2_6S', description='The 6s clips recorded from sm02 devices, which are used for the projects', instructions='')


In [5]:
# view the clip annotations
for clip_annotation in gn_sample.clip_annotations:
    clip = clip_annotation.clip
    recording = clip.recording
    print(
        f"* Recording {recording.path} [from "
        f"{clip.start_time:.3f}s to {clip.end_time:.3f}s]"
    )
    print(
        f"\t{len(clip_annotation.sound_events)} sound event annotations found"
    )
    for annotation in clip_annotation.sound_events:
        sound_event = annotation.sound_event
        print(f"\t+ Sound event ")
        for tag in annotation.tags:
            print(f"\t\t- {tag}")
    print("")

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_01_2021\GN1_20210114_110000.wav [from 120.000s to 126.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_04_2021\GN1_20210401_170000.wav [from 96.000s to 102.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_02_2021\GN1_20210228_150000.wav [from 156.000s to 162.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_03_2021\GN1_20210325_190000.wav [from 198.000s to 204.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_01_2021\GN1_20210105_120000.wav [from 18.000s to 24.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_03_2021\GN1_20210324_060000.w

In [14]:
# save the annotation project
io.save(gn_sample, "gn_sample.json")

In [18]:
# 2) select the annotated clips

In [10]:
# create functions
def load_annotation_file(annotation_file):
    return io.load(annotation_file, type="annotation_project")


def has_issue(task):
    for badge in task.status_badges:
        if badge.state == data.AnnotationState.rejected:
            return True

    return False


def is_complete(task):
    for badge in task.status_badges:
        if badge.state == data.AnnotationState.completed:
            return True

    return False


def select_clips(annotation_project, complete=None, issue=None):
    tasks = [task for task in annotation_project.tasks]

    if complete is not None:
        tasks = [task for task in tasks if is_complete(task) == complete]

    if issue is not None:
        tasks = [task for task in tasks if has_issue(task) == issue]

    clips = {task.clip.uuid for task in tasks}
    return [
        clip_annotation
        for clip_annotation in annotation_project.clip_annotations
        if clip_annotation.clip.uuid in clips
    ]

path = "gn_sample.json"
annotation_project = load_annotation_file(path)
complete_clips = select_clips(annotation_project, complete=True, issue=False)

In [12]:
# view the completely annotated clips
for clip_annotation in complete_clips:
    clip = clip_annotation.clip
    recording = clip.recording
    print(
        f"* Recording {recording.path} [from "
        f"{clip.start_time:.3f}s to {clip.end_time:.3f}s]"
    )

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_01_2021\GN1_20210114_110000.wav [from 120.000s to 126.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_04_2021\GN1_20210401_170000.wav [from 96.000s to 102.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_02_2021\GN1_20210228_150000.wav [from 156.000s to 162.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_03_2021\GN1_20210325_190000.wav [from 198.000s to 204.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_01_2021\GN1_20210105_120000.wav [from 18.000s to 24.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_03_2021\GN1_20210324_060000.wav [from 138.000s to 144.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM02 (GN1)\2021\SM02_02_2021\GN1_20210201_140000.wav [from 300.000s to 306.0

In [14]:
len(complete_clips)

1000

In [28]:
# 3) Extract features
 # Load latest annotations
annotation_project = io.load("gn_sample.json")

In [16]:
# Load the birdnet model into the notebook!
model = BirdNET.load()

In [18]:
# as all 990 clips have been completely annotated, no need to remove any clips from the original json
selected_clip_annotations = annotation_project.clip_annotations[:1000]

In [20]:
# check the audio dir and path
prefix = Path.home() / "OneDrive" / "桌面" / "seychelles_parrot_data"
print(f"Prefix exists: {prefix.exists()}")
audio_dir = Path("SM02 (GN1)")
print(f"audio dir: {audio_dir.exists()}")

Prefix exists: True
audio dir: False


In [22]:
# fix the path

for clip_annotation in selected_clip_annotations:
    clip = clip_annotation.clip
    recording = clip.recording

    # Only fix the path if not previously done
    recording.path = Path(str(recording.path).replace("妗岄潰","桌面"))

    if not recording.path.exists():
        raise ValueError(f"File not found {recording.path}")

In [24]:
# view the infos using an example clip
example_clip = selected_clip_annotations[0].clip

print("Clips contain info about when they start and end: ", clip.start_time, clip.end_time)

example_outputs = model.process_clip(example_clip)

print("The output contains two processed clips", len(example_outputs))

example_processed_clip = example_outputs[0]

print("Processed clips might be a subclip of the input: ", example_processed_clip.clip.start_time, example_processed_clip.clip.end_time) 

print("Processed clips can contain extracted features. In this case: ", len(example_processed_clip.features))

print("The first feature:  ", example_processed_clip.features[0])

Clips contain info about when they start and end:  144.0 150.0
The output contains two processed clips 2
Processed clips might be a subclip of the input:  120.0 123.0
Processed clips can contain extracted features. In this case:  1024
The first feature:   name='BirdNET_0' value=0.0012620389461517334


In [26]:
# Process all clips in a loop
def extract_tags_info(tags):
    return {tag.value: True for tag in tags}


def extract_task_info(annotation_task):
    ready = False
    issues = False

    for badge in annotation_task.status_badges:
        if badge.state == data.AnnotationState.completed:
            ready = True
            continue

        if badge.state == data.AnnotationState.rejected:
            issues = True
            continue

    return {"ready": ready, "issues": issues}


def extract_clip_annotation_info(clip_annotation):
    return {
        "annotation_id": str(clip_annotation.uuid),
        **extract_tags_info(clip_annotation.tags),
    }


def extract_feature_info(processed_clip):
    return {
        "path": processed_clip.clip.recording.path,
        "clip_id": processed_clip.clip.uuid,
        "start_time": processed_clip.clip.start_time,
        "end_time": processed_clip.clip.end_time,
        **{feature.name: feature.value for feature in processed_clip.features},
    }


# Process all clips in a loop
outputs = []
for clip_annotation in tqdm(selected_clip_annotations):
    clip = clip_annotation.clip

    # Its a good idea to wrap this processing bit in a try-except block
    # so that we can safely skip the clips that failed to process.
    try:
        processed_clips = model.process_clip(clip)

        # NOTE: If the clip is long the model will split it into multiple clips
        # and return a list of processed clips.
        for processed_clip in processed_clips:
            outputs.append(
                {
                    **extract_feature_info(processed_clip),
                    **extract_clip_annotation_info(clip_annotation),
                }
            )

    except Exception as e:
        print(f"Failed to process clip: {clip}. Error: {e}")
        continue

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:00<00:00,  5.55it/s]


In [30]:
# creat a pandas dataframe 
df = pd.DataFrame(outputs)
df

,path,clip_id,start_time,end_time,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,...,unknown,bird,whistle,bulbul,drops,human,rain,frog,chat,alarm
0,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,7e26bebe-5511-49be-87a8-cc174f92c5dc,120.0,123.0,0.001262,0.028853,0.172439,0.111442,0.867608,0.138024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,60065d14-5c9d-4017-bc35-8f592a4b3132,123.0,126.0,0.050983,0.168289,0.140707,0.065963,0.818231,0.686815,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,90a94494-adfb-476c-8f85-cdecce13e4fc,96.0,99.0,0.137913,0.000000,0.102183,0.764601,0.246049,0.211196,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,b70003e9-492b-4cd3-8174-1ab780d35f1a,99.0,102.0,0.045321,0.000000,0.330488,0.586221,0.704817,0.070498,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,e356f2c7-c243-4e01-a195-a55e2f7e3365,156.0,159.0,0.571633,0.000000,0.189225,0.332366,0.719348,0.475060,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,428ba83f-1d8a-4d18-b491-7e2874af4fd1,201.0,204.0,0.153800,0.081599,0.228782,1.088008,0.450123,0.262619,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,9142bd05-7012-49d6-b449-ee9c8d3880fb,150.0,153.0,0.000000,0.272483,0.204294,0.203722,0.529087,0.328534,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,e9409adc-d75c-4fc4-a176-a5419aeb21d2,153.0,156.0,0.000000,0.170321,0.169106,0.296746,0.471260,0.653617,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,768c417c-d755-4426-8c9a-de4742cf530c,144.0,147.0,0.077993,0.000000,0.286379,0.487577,0.198941,0.338931,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df.to_csv("full_gn_features.csv")

In [48]:
# compute the average features for two subclips
full_gn_features = pd.read_csv('full_gn_features.csv', low_memory=False)
full_gn_features.drop(columns = ['clip_id',	'Unnamed: 0'])

,path,start_time,end_time,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,...,unknown,bird,whistle,bulbul,drops,human,rain,frog,chat,alarm
0,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,120.0,123.0,0.001262,0.028853,0.172439,0.111442,0.867608,0.138024,0.558496,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,123.0,126.0,0.050983,0.168289,0.140707,0.065963,0.818231,0.686815,0.534349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,96.0,99.0,0.137913,0.000000,0.102183,0.764601,0.246049,0.211196,0.905824,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,99.0,102.0,0.045321,0.000000,0.330488,0.586221,0.704817,0.070498,0.227793,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,156.0,159.0,0.571633,0.000000,0.189225,0.332366,0.719348,0.475060,0.451623,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,201.0,204.0,0.153800,0.081599,0.228782,1.088008,0.450123,0.262619,1.065952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,150.0,153.0,0.000000,0.272483,0.204294,0.203722,0.529087,0.328534,0.194385,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,153.0,156.0,0.000000,0.170321,0.169106,0.296746,0.471260,0.653617,0.042280,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,144.0,147.0,0.077993,0.000000,0.286379,0.487577,0.198941,0.338931,0.420429,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
full_gn_features.columns

Index(['Unnamed: 0', 'path', 'clip_id', 'start_time', 'end_time', 'BirdNET_0',
       'BirdNET_1', 'BirdNET_2', 'BirdNET_3', 'BirdNET_4',
       ...
       'unknown', 'bird', 'whistle', 'bulbul', 'drops', 'human', 'rain',
       'frog', 'chat', 'alarm'],
      dtype='object', length=1043)

In [56]:
# Define columns that are categorical (factors)
categorical_columns = ['path', 'parrot','nothing','rain or wind', 'bird', 'unknown', 'whistle', 'bulbul', 'drops', 'human', 'rain',
       'frog', 'chat', 'alarm']  # Add any other categorical columns here

# Identify numeric columns for averaging
numeric_columns = full_gn_features.select_dtypes(include=[float, int]).columns

# Remove categorical and time columns from numeric columns list if they are included
numeric_columns = [col for col in numeric_columns if col not in categorical_columns + ['start_time', 'end_time','annotation_id']]

# Group by 'annotation_id' and compute the average of each numeric feature column
fused_features_numeric = full_gn_features.groupby('annotation_id')[numeric_columns].mean().reset_index()

# For categorical columns, we can keep the first instance (assuming consistency)
# Exclude 'annotation_id' here to avoid duplication
fused_features_categorical = full_gn_features.groupby('annotation_id')[categorical_columns].first().reset_index()

# For start_time and end_time,custom logic
start_times = full_gn_features.groupby('annotation_id')['start_time'].first().reset_index()
end_times = full_gn_features.groupby('annotation_id')['end_time'].last().reset_index()

# Merge numeric, categorical, and time DataFrames
fused_features = fused_features_numeric.merge(fused_features_categorical, on='annotation_id')
fused_features = fused_features.merge(start_times, on='annotation_id')
fused_features = fused_features.merge(end_times, on='annotation_id')
fused_features.drop(columns = ['Unnamed: 0'])

,annotation_id,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,BirdNET_7,BirdNET_8,...,whistle,bulbul,drops,human,rain,frog,chat,alarm,start_time,end_time
0,001743bf-8abc-44c2-88f9-1747de3c0397,0.097621,0.669875,0.315825,0.085709,0.655792,0.226732,0.922827,0.322524,0.104912,...,None,None,None,None,None,None,None,None,258.0,264.0
1,0032eb76-59eb-42c2-a76b-482d3195b419,0.123475,0.777490,0.083553,0.074976,0.253921,0.014245,0.940176,0.146392,0.215682,...,None,None,None,None,None,None,None,None,168.0,174.0
2,00604ef9-5173-44eb-b25d-9edb2d27b3dd,0.156897,0.142796,0.049727,0.105337,0.266970,0.141995,1.238247,0.045868,0.443223,...,None,None,None,None,None,None,None,None,252.0,258.0
3,012b59b9-3a07-4d64-aac6-49cf8c61a96d,0.306011,0.163392,0.340511,0.660456,0.630914,0.385653,0.407340,0.162842,0.341946,...,None,None,None,None,None,None,None,None,276.0,282.0
4,01861604-ae2d-4d8e-901f-ed72557d248c,0.040909,0.080098,0.280291,0.443766,0.527499,0.191382,0.712419,0.240354,0.196569,...,None,None,None,None,None,None,None,None,36.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,ff75df6c-09b0-46dc-92d0-9bba756dc1ef,0.008055,0.225173,0.063461,0.275016,0.410493,0.651830,0.000000,0.359898,0.275843,...,None,None,None,None,None,None,None,None,300.0,306.0
996,ff9acd23-1200-42c5-8625-783030162a35,0.686403,0.015064,0.155991,0.221385,1.116837,0.744202,0.321928,0.017607,0.196264,...,None,None,None,None,None,None,None,None,144.0,150.0
997,fff16b70-9563-4283-b210-3dd7cf5a6bd6,0.065828,0.508203,0.045878,0.467270,0.246571,0.112591,0.981714,0.154203,0.432141,...,None,None,None,None,None,None,None,None,120.0,126.0
998,fffcec0d-28ae-4ec5-a7fa-9b500837c1bf,0.202528,0.219135,0.145106,0.043340,1.469910,0.036213,0.216147,0.138415,0.048050,...,None,None,None,None,None,None,None,None,198.0,204.0


In [58]:
# save the fused features in to csv
fused_features.to_csv("full_gn_features.csv")

In [60]:
fused_features.columns

Index(['annotation_id', 'Unnamed: 0', 'BirdNET_0', 'BirdNET_1', 'BirdNET_2',
       'BirdNET_3', 'BirdNET_4', 'BirdNET_5', 'BirdNET_6', 'BirdNET_7',
       ...
       'whistle', 'bulbul', 'drops', 'human', 'rain', 'frog', 'chat', 'alarm',
       'start_time', 'end_time'],
      dtype='object', length=1042)

In [68]:
# dropping unwanted columns and add site information for the ml development
ml_features = fused_features.drop(columns = ['Unnamed: 0','whistle','bulbul','bird','nothing', 'unknown', 'rain or wind', 'drops', 'human', 'rain', 'frog', 'chat', 'alarm'])
ml_features['site']='gn'

In [70]:
ml_features

,annotation_id,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,BirdNET_7,BirdNET_8,...,BirdNET_1019,BirdNET_1020,BirdNET_1021,BirdNET_1022,BirdNET_1023,path,parrot,start_time,end_time,site
0,001743bf-8abc-44c2-88f9-1747de3c0397,0.097621,0.669875,0.315825,0.085709,0.655792,0.226732,0.922827,0.322524,0.104912,...,0.450701,0.482047,0.166951,0.039508,1.630975,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,258.0,264.0,gn
1,0032eb76-59eb-42c2-a76b-482d3195b419,0.123475,0.777490,0.083553,0.074976,0.253921,0.014245,0.940176,0.146392,0.215682,...,0.399222,0.714610,0.202627,0.097467,1.186374,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,168.0,174.0,gn
2,00604ef9-5173-44eb-b25d-9edb2d27b3dd,0.156897,0.142796,0.049727,0.105337,0.266970,0.141995,1.238247,0.045868,0.443223,...,0.806098,0.468591,0.373682,0.000000,1.132183,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,252.0,258.0,gn
3,012b59b9-3a07-4d64-aac6-49cf8c61a96d,0.306011,0.163392,0.340511,0.660456,0.630914,0.385653,0.407340,0.162842,0.341946,...,0.311247,0.331802,0.204257,0.006983,0.843150,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,276.0,282.0,gn
4,01861604-ae2d-4d8e-901f-ed72557d248c,0.040909,0.080098,0.280291,0.443766,0.527499,0.191382,0.712419,0.240354,0.196569,...,0.468476,0.309960,0.166623,0.000000,0.755087,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,36.0,42.0,gn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,ff75df6c-09b0-46dc-92d0-9bba756dc1ef,0.008055,0.225173,0.063461,0.275016,0.410493,0.651830,0.000000,0.359898,0.275843,...,0.516895,0.292650,0.740552,0.170354,0.748197,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,300.0,306.0,gn
996,ff9acd23-1200-42c5-8625-783030162a35,0.686403,0.015064,0.155991,0.221385,1.116837,0.744202,0.321928,0.017607,0.196264,...,0.535561,0.152875,0.150754,0.273586,1.220475,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,144.0,150.0,gn
997,fff16b70-9563-4283-b210-3dd7cf5a6bd6,0.065828,0.508203,0.045878,0.467270,0.246571,0.112591,0.981714,0.154203,0.432141,...,1.002185,0.514526,0.291000,0.000000,1.201089,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,120.0,126.0,gn
998,fffcec0d-28ae-4ec5-a7fa-9b500837c1bf,0.202528,0.219135,0.145106,0.043340,1.469910,0.036213,0.216147,0.138415,0.048050,...,0.523054,0.520617,1.365951,0.149450,1.646158,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,198.0,204.0,gn


In [72]:
ml_features.to_csv("ml_gn_features.csv")